In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras

%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt

import os
import time

In [2]:
""" here "drcaps" is file caps-dr-layer_v2.py """
from drcaps import *

In [3]:
c1=tf.keras.layers.Conv2D(16,kernel_size=5,strides=1,padding='valid',activation='relu')
c2=tf.keras.layers.Conv2D(32,kernel_size=9,strides=1,padding='valid',activation='relu')
bn1=tf.keras.layers.BatchNormalization()
bn2=tf.keras.layers.BatchNormalization()
last=Capsule(10,16)

In [4]:
model_input = keras.Input(shape=(28,28,1))
x=c1(model_input)
x=bn1(x,training=True)
x=c2(x)
x=bn2(x,training=True)
x=tf.reshape(x,[-1,16*32,16])
x=squash(x) # squash function needed to be applied.
x=last(x)
#x=tf.cast(x,tf.float32)
x=safe_norm(x, axis=2)
model_output = x

In [5]:

model = keras.Model(model_input, model_output)

In [6]:
adam = tf.keras.optimizers.Adam(learning_rate=0.0001) 
model.compile(loss=margin_loss,
              optimizer=adam,
              metrics=tf.keras.metrics.CategoricalAccuracy())
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 28, 28, 1)]       0         
                                                                 
 conv2d (Conv2D)             (None, 24, 24, 16)        416       
                                                                 
 batch_normalization (BatchN  (None, 24, 24, 16)       64        
 ormalization)                                                   
                                                                 
 conv2d_1 (Conv2D)           (None, 16, 16, 32)        41504     
                                                                 
 batch_normalization_1 (Batc  (None, 16, 16, 32)       128       
 hNormalization)                                                 
                                                                 
 tf.reshape (TFOpLambda)     (None, 512, 16)           0     

In [7]:
# data loading in appropriate formate

mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

# Add a channels dimension
x_train = x_train[..., tf.newaxis].astype("float32")
x_test = x_test[..., tf.newaxis].astype("float32")

y_train=tf.keras.utils.to_categorical(y_train)
y_test=tf.keras.utils.to_categorical(y_test)
     

11490434/11490434 [==============================] - 0s 0us/step


In [ ]:
model.fit(x_train, y_train, batch_size=32,epochs=5,validation_split=0.2)

Epoch 1/5
 146/1500 [=>............................] - ETA: 10:10 - loss: 0.1728 - categorical_accuracy: 0.8587